# Notebook of the project of Physics of Complex Systems #

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import seaborn as sns

Firstly, I try to reproduce the mean-field approach. The resulting system of ODEs is the following:
$$
    \frac{\partial x}{\partial t} = \alpha x - \gamma xy , \;\;\;
    \frac{\partial y}{\partial t} = \lambda + \nu xy - \sigma y
$$
where $x$ is the pathogen concentration and $y$ lymphocytes'. \
The parameters represent:
- $\alpha$ : proliferation rate of the pathogen; 
- $\gamma$ : destruction rate of the pathogen by the lymphocytes; 
- $\lambda$ : birth rate of the lymphocyte; 
- $\nu$ : duplication rate of the lymphocyte when encountering the pathogen; 
- $\sigma$ : death rate of the lymphocyte. 


## Mean Field Approach ##

In [ ]:
# set initial conditions
z0 = np.array([1,np.random.poisson(lam=1)])    # z=[x,y]

# parameters initialization (I use the parameters listed in the caption of Figure 3)
par = {
    'alpha': 1.6,
    'gamma': 0.01,   # questo sarebbe 0.01
    'lambda': 0.6,
    'nu': 0.001,
    'sigma': 0.01
}

# In which regime are we?
print(f'In which regime are we? {'First regime:(0,y_0) unstable, (x^*,y^*) stable' \
    if par['alpha']>(par['lambda']*par['gamma']/par['sigma']) else '\text{Second regime:}(0,y_0) \text{stable}, (x^*,y^*) \text{unstable}'}')

def system(z:np.ndarray ,t:np.ndarray ,par:dict):
    
    x, y = z
    dxdt = par['alpha']*x - par['gamma']*x*y
    dydt = par['lambda'] + par['nu']*x*y - par['sigma']*y
    return [dxdt,dydt]

# set the time span 
#t_s = np.linspace(0,int(1e4),int(1e5))
t_s = np.linspace(0,500,10000)      # credo sia questo il time step che hanno usato

# Solve
X = odeint(system,z0,t_s,args=(par,))[:,0]
Y = odeint(system,z0,t_s,args=(par,))[:,1]

# Normalization

# Solution of the X ODE
#plt.figure(figsize=(12,8))
plt.plot(t_s, X, c='orange', label='x(t)',alpha=0.7)
plt.title('Solution of the ODEs in time under mean-field approximation')
#plt.plot(t_s,Y,c='magenta',label='y(t)',alpha=0.7)
plt.xlabel('Time t')
plt.ylabel('Concentration')
plt.xscale('log')
plt.grid(True, which="both",alpha=0.4,linestyle='--')
plt.legend()
plt.tight_layout()
plt.show()


Now I recreate the trajectory of the solutions of the ODE

In [ ]:
# set initial conditions
z0 = np.array([1,np.random.poisson(lam=1)])    # z=[x,y]

# parameters initialization (I use the parameters listed in the caption of Figure 3)
par = {
    'alpha': 1.2,
    'gamma': 0.1,   # questo sarebbe 0.01
    'lambda': 0.4,
    'nu': 0.004,
    'sigma': 0.1,
    'Dx': 0.,       # diffusion of x
    'Dy': 0.        # diffusion of y
}

# Define some meaningful quantities for the analysis
y0 = par['lambda']/par['sigma']
y_star = par['alpha']/par['gamma']
x_star = (par['sigma']-par['lambda']*par['gamma']/par['alpha'])/par['nu']

# In which regime are we?
print(f'In which regime are we? {'First regime:(0,y_0) unstable, (x^*,y^*) stable' \
    if par['alpha']>(par['lambda']*par['gamma']/par['sigma']) else '\text{Second regime:}(0,y_0) \text{stable}, (x^*,y^*) \text{unstable}'}')

def system(z:np.ndarray ,t:np.ndarray ,par:dict):
    
    x, y = z
    dxdt = par['alpha']*x - par['gamma']*x*y
    dydt = par['lambda'] + par['nu']*x*y - par['sigma']*y
    return [dxdt,dydt]

# set the time span 
#t_s = np.linspace(0,int(1e4),int(1e5))
t_s = np.linspace(0,200,10000)      # credo sia questo il time step che hanno usato

# Solve
X = odeint(system,z0,t_s,args=(par,))[:,0]
Y = odeint(system,z0,t_s,args=(par,))[:,1]

# Normalization


# Trajectory of the solutions over time
plt.plot(X, Y, c='orange', label='y(t)',alpha=0.7)
plt.title('Trajectory of the solution of the ODEs in mean-field approximation')
plt.scatter(0,y0, c='r',marker='^',label=r'$(0,y_0)$')
plt.scatter(x_star,y_star, c='r',marker='*',label=r'$(x^*,y^*)$')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True, which="both",alpha=0.4,linestyle='--')
plt.legend()
plt.tight_layout()
plt.show()


## Lattice ##
The interactions between pathogens and lymphocytes are described by the following reactions:
$$
\begin{gather}
    X \xrightarrow{\alpha} X + X \\
    \emptyset \xrightarrow{\lambda} Y \\
    X + Y \xrightarrow{\gamma} Y  \\
    X + Y \xrightarrow{\nu} X + 2Y \\
    Y \xrightarrow{\sigma} \emptyset .
\end{gather}
$$

### 1D ##
Firstly, I try a toy model of a 1D lattice with 100 cells and no diffusion (mean-field approximation):

In [ ]:
def check_sign(cell):
    # If in any cell x or y are negative, set it to zero
    if cell[0] < 0:
        cell[0] = 0
        
    if cell[1] < 0:
        cell[1] = 0

'''
par = {
    'alpha': 1.2,
    'gamma': 0.1,  
    'lambda': 0.4,
    'nu': 0.004,
    'sigma': 0.1,
    'Dx': 0.,
    'Dy': 0.
}
'''

par = {
    'alpha': 1.2,
    'gamma': 0.1,   # questo sarebbe 0.01
    'lambda': 0.4,
    'nu': 0.004,
    'sigma': 0.1,
    'Dx': 0.,       # diffusion of x
    'Dy': 0.        # diffusion of y
}

# In which regime are we?
print(f'In which regime are we? {'First regime:(0,y_0) unstable, (x^*,y^*) stable' \
    if par['alpha']>(par['lambda']*par['gamma']/par['sigma']) else '\text{Second regime:}(0,y_0) \text{stable}, (x^*,y^*) \text{unstable}'}')

# Define the lattice
lattice = np.zeros((100,2))

# Inizialization    (x_i,y_0) for each site, with x_i = 3, y_0 = \lambda / \sigma
lattice[:,0] = 3    
lattice[:,1] = par['lambda'] / par['sigma']

# Time steps
time_steps = 10000

# Initialize arrays to contain the mean of x and y
X = np.zeros(time_steps)
Y = np.zeros(time_steps)

# Loop
for t in range(time_steps):
    for i in range(len(lattice)):
        lattice[i,0] += np.random.poisson(par['alpha']*lattice[i,0])                # proliferation of the pathogen
        lattice[i,1] += np.random.poisson(par['lambda'])                            # birth of lymphocyte
        lattice[i,0] -= np.random.poisson(par['gamma']*lattice[i,0]*lattice[i,1])   # lymphocyte kills pathogen
        check_sign(lattice[i,:])                                                    # check the sign only after there's a subtraction
        lattice[i,1] += np.random.poisson(par['nu']*lattice[i,0]*lattice[i,1])      # lymphocyte duplication
        lattice[i,1] -= np.random.poisson(par['sigma']*lattice[i,1])
        check_sign(lattice[i,:])

    # calculate the mean of x and y, so to compare it to the mean-field result
    X[t] = np.mean(lattice[:,0])
    Y[t] = np.mean(lattice[:,1])
    
    #print(*lattice)                                                                # the asterisk in the print is used to print horizontally

# plot the evolution of x(t)
#plt.figure(figsize=(12,8))
plt.plot(np.linspace(0,500,time_steps), X, c='orange', label='x(t)',alpha=0.7)
plt.title('Averaged zero-dimensional stochastic counterpart of the mean-field results')
#plt.plot(t_s,Y,c='magenta',label='y(t)',alpha=0.7)
plt.xlabel('Time t')
plt.ylabel('Concentration')
plt.xscale('log')
plt.grid(True, which="both",alpha=0.4,linestyle='--')
plt.legend()
plt.tight_layout()
plt.show()
